<a href="https://colab.research.google.com/github/earo12/ETL_PepsiCo/blob/dev_barnch/ETL_PepsiCo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# First we need to import some necessary libraries
# Since the PDF doesn't mention what kind of source we're going to need, I'm going to create them with PySpark
# It can be done with other data tools such as Pandas, or a SQL database and even a NoSQL database such as Mongo
# However, I think that the standard way to manipulate data is with Spark so I'm going to use it in this project in particular
# I'm writing this code in Colab, but actually, it can be run in practically every enviroment that incorporates Spark
# If your environment doesn't have installed Spark, please make sure to run this line first:
!pip install pyspark

In [3]:
# Now we can import the necessary libraries:

from pyspark.sql import SparkSession # Session from Spark, it endes up when we close this notebook
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType # Fields recquired for the three tables needed in the challenge

In [4]:
# We can create the Spark Session as sp:

sp = SparkSession.builder.master("local[*]").appName("PepsiCoCase").getOrCreate()
